In [2]:
import pandas as pd
import os

In [2]:
# pip install tensorflow==2.14.0

In [3]:
import tensorflow
tensorflow.__version__

2024-03-09 02:14:25.743411: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 02:14:25.743495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 02:14:25.743530: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 02:14:25.752969: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 02:14:31.456318: W tensorflow/compiler/

'2.14.0'

In [4]:
train_data = pd.read_csv('./train/train.csv')

In [5]:
%%capture
train_data.drop(['Name', 'RescuerID'], axis = 1, inplace = True)
train_data['AdoptionSpeed'][train_data['AdoptionSpeed']<=2] = 1
train_data['AdoptionSpeed'][train_data['AdoptionSpeed']>2] = 0
tr_data = train_data.copy(deep=True)
tr_data.dropna(inplace = True)
tr_data.reset_index(inplace = True)
train_text = tr_data[['Description','AdoptionSpeed']]

img_files = os.listdir('./train_images/')
files = [img for img in img_files if img.endswith('-1.jpg')]
img_names = set(imgname[:-6] for imgname in files)
img_omit = set(tr_data['PetID']) - img_names
tr_data = tr_data[~tr_data.PetID.isin(img_omit)]
tr_img = tr_data[['PetID','AdoptionSpeed']]
def generate_image_name(pet_id):
    return f"{pet_id}-1.jpg"
tr_img['ImageName'] = tr_img['PetID'].apply(generate_image_name)

tr_text = tr_data[['Description', 'AdoptionSpeed']]

tr_tab = tr_data.drop(['Description', 'PetID', 'index' ,'Breed2'], axis = 1)

In [6]:
breed_df = pd.read_csv('./PetFinder-BreedLabels.csv')
breed_df.drop(['Type'], axis = 1, inplace = True)
merged_data = tr_tab.merge(breed_df, left_on = 'Breed1', right_on = 'BreedID', how = 'left').drop(
    ['Breed1', 'BreedID'], axis = 1)
breed_counts_x = merged_data['BreedName'].value_counts()
replace_breeds_x = breed_counts_x[breed_counts_x <= 100].index
merged_data['BreedName'] = merged_data['BreedName'].apply(lambda x: 'Others' if x in replace_breeds_x else x)


In [7]:
type_mapping_type = {1: 'Dog', 2: 'Cat'}
merged_data['Type'] = merged_data['Type'].map(type_mapping_type)

type_mapping_gender = {1: 'Male', 2: 'Female', 3: 'Mixed'}
merged_data['Gender'] = merged_data['Gender'].map(type_mapping_gender)

type_mapping_color = {1: 'Black', 2: 'Brown', 3: 'Golden', 4: 'Yellow', 5: 'Cream', 6: 'Gray', 7: 'White', 0: 'NA' }
merged_data['Color1'] = merged_data['Color1'].map(type_mapping_color)
merged_data['Color2'] = merged_data['Color2'].map(type_mapping_color)
merged_data['Color3'] = merged_data['Color3'].map(type_mapping_color)

type_mapping_vaccination = {1: 'Yes', 2: 'No', 3: 'NotSure'}
merged_data['Vaccinated'] = merged_data['Vaccinated'].map(type_mapping_vaccination)

type_mapping_Dewormed = {1: 'Yes', 2: 'No', 3: 'NotSure'}
merged_data['Dewormed'] = merged_data['Dewormed'].map(type_mapping_Dewormed)

type_mapping_Sterilized = {1: 'Yes', 2: 'No', 3: 'NotSure'}
merged_data['Sterilized'] = merged_data['Sterilized'].map(type_mapping_Sterilized)


In [8]:
state_df = pd.read_csv('./state_labels.csv')
merged_data = merged_data.merge(state_df, how = 'left', left_on = 'State', 
                                right_on = 'StateID').drop(['StateID', 'State'], axis = 1)
states = merged_data.StateName.value_counts()
statelst = states[states<150].index.tolist()

In [9]:
merged_data['BreedName'] = merged_data['BreedName'].apply(lambda x: 'Others' if x in replace_breeds_x else x)
merged_data['StateName'] = merged_data['StateName'].apply(lambda x: 'Others' if x in statelst else x)

data_dummies = pd.get_dummies(merged_data[['Type', 'Gender', 'Vaccinated', 'Dewormed', 'Sterilized', 'BreedName', 'StateName']])

for color_col in ['Color1', 'Color2', 'Color3']:
    color_dummies = pd.get_dummies(merged_data[color_col], prefix='Color')
    if color_col == 'Color1':
        factor = 3
    elif color_col == 'Color2': 
        factor = 2
    else: factor = 1
    data_dummies = data_dummies.add(color_dummies.multiply(factor), fill_value=0)


In [10]:
final_tab_data = pd.concat([merged_data, data_dummies], axis = 1)
final_tab_data.drop(['Type', 'Gender', 'Vaccinated', 'Dewormed', 'Sterilized', 
                      'BreedName', 'StateName', 'Color1', 'Color2', 'Color3'], axis =1, inplace = True)


In [11]:
final_tab_data.head()

,Age,MaturitySize,FurLength,Health,Quantity,Fee,VideoAmt,PhotoAmt,AdoptionSpeed,BreedName_Calico,...,StateName_Pulau Pinang,StateName_Selangor,Sterilized_No,Sterilized_NotSure,Sterilized_Yes,Type_Cat,Type_Dog,Vaccinated_No,Vaccinated_NotSure,Vaccinated_Yes
0,3,1,1,1,1,100,0,1.0,1,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,2,2,1,1,0,0,2.0,1,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,2,2,1,1,0,0,7.0,0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,4,2,1,1,1,150,0,8.0,1,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,2,1,1,1,0,0,3.0,1,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [12]:
from sklearn.preprocessing import Normalizer
normalscaler = Normalizer()
tab_data_X = normalscaler.fit_transform(final_tab_data)

In [13]:
tab_data_X[:2]

array([[0.02994615, 0.00998205, 0.00998205, 0.00998205, 0.00998205,
        0.99820485, 0.        , 0.00998205, 0.00998205, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00998205, 0.        , 0.02994615, 0.        ,
        0.        , 0.        , 0.        , 0.00998205, 0.0199641 ,
        0.        , 0.00998205, 0.        , 0.        , 0.        ,
        0.00998205, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00998205, 0.00998205,
        0.        , 0.        , 0.00998205, 0.        , 0.00998205,
        0.        , 0.        ],
       [0.16439899, 0.32879797, 0.32879797, 0.16439899, 0.16439899,
        0.        , 0.        , 0.32879797, 0.16439899, 0.        ,
        0.        , 0.16439899, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.     

In [14]:
# from sklearn.model_selection import train_test_split

# X_train_structured_std, X_test_structured_std, y_train_std, y_test_std = train_test_split(
#     final_tab_data.drop('AdoptionSpeed', axis = 1), tab_data_y, test_size = 0.25, random_state = 42)

In [15]:
tr_text

,Description,AdoptionSpeed
0,Nibble is a 3+ month old ball of cuteness. He ...,1
1,I just found it alone yesterday near my apartm...,1
2,Their pregnant mother was dumped by her irresp...,0
3,"Good guard dog, very alert, active, obedience ...",1
4,This handsome yet cute boy is up for adoption....,1
...,...,...
14975,I have 4 kittens that need to be adopt urgentl...,1
14976,Serato(female cat- 3 color) is 4 years old and...,0
14977,"Mix breed, good temperament kittens. Love huma...",0
14978,she is very shy..adventures and independent..s...,0


# AllText unimodal

In [16]:
tr_data.drop(['index','PetID'], axis = 1, inplace = True)

In [17]:
tr_data.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'Description', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [18]:
breed_df = pd.read_csv('./PetFinder-BreedLabels.csv')
breed_df.drop(['Type'], axis = 1, inplace = True)
merged_data = tr_data.merge(breed_df, left_on = 'Breed1', right_on = 'BreedID', how = 'left').drop(
    ['Breed1', 'BreedID'], axis = 1)
merged_data.rename({'BreedName': 'Breed1'}, axis =1, inplace = True)
merged_data = merged_data.merge(breed_df, left_on = 'Breed2', right_on = 'BreedID', how = 'left').drop(
    ['Breed2', 'BreedID'], axis = 1)
merged_data.rename({'BreedName': 'Breed2'}, axis =1, inplace = True)

In [19]:
merged_data

,Type,Age,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,...,Health,Quantity,Fee,State,VideoAmt,Description,PhotoAmt,AdoptionSpeed,Breed1,Breed2
0,2,3,1,1,7,0,1,1,2,2,...,1,1,100,41326,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,1,Tabby,NaN
1,2,1,1,1,2,0,2,2,3,3,...,1,1,0,41401,0,I just found it alone yesterday near my apartm...,2.0,1,Domestic Medium Hair,NaN
2,1,1,1,2,7,0,2,2,1,1,...,1,1,0,41326,0,Their pregnant mother was dumped by her irresp...,7.0,0,Mixed Breed,NaN
3,1,4,2,1,2,0,2,1,1,1,...,1,1,150,41401,0,"Good guard dog, very alert, active, obedience ...",8.0,1,Mixed Breed,NaN
4,1,1,1,1,0,0,2,1,2,2,...,1,1,0,41326,0,This handsome yet cute boy is up for adoption....,3.0,1,Mixed Breed,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,2,2,3,1,0,0,2,2,2,2,...,1,4,0,41326,0,I have 4 kittens that need to be adopt urgentl...,3.0,1,Domestic Short Hair,NaN
14636,2,60,3,1,4,7,2,2,1,1,...,1,2,0,41326,0,Serato(female cat- 3 color) is 4 years old and...,3.0,0,Domestic Medium Hair,Domestic Long Hair
14637,2,2,3,5,6,7,3,2,2,1,...,1,5,30,41326,0,"Mix breed, good temperament kittens. Love huma...",5.0,0,Domestic Medium Hair,Domestic Short Hair
14638,2,9,2,4,7,0,1,1,1,1,...,1,1,0,41336,0,she is very shy..adventures and independent..s...,3.0,0,Domestic Short Hair,NaN


In [20]:
type_mapping_type = {1: 'Dog', 2: 'Cat'}
merged_data['Type'] = merged_data['Type'].map(type_mapping_type)

type_mapping_gender = {1: 'Male', 2: 'Female', 3: 'Mixed'}
merged_data['Gender'] = merged_data['Gender'].map(type_mapping_gender)

type_mapping_color = {1: 'Black', 2: 'Brown', 3: 'Golden', 4: 'Yellow', 5: 'Cream', 6: 'Gray', 7: 'White', 0: 'NA' }
merged_data['Color1'] = merged_data['Color1'].map(type_mapping_color)
merged_data['Color2'] = merged_data['Color2'].map(type_mapping_color)
merged_data['Color3'] = merged_data['Color3'].map(type_mapping_color)

type_mapping_vaccination = {1: 'Yes', 2: 'No', 3: 'Not Sure'}
merged_data['Vaccinated'] = merged_data['Vaccinated'].map(type_mapping_vaccination)

type_mapping_Dewormed = {1: 'Yes', 2: 'No', 3: 'Not Sure'}
merged_data['Dewormed'] = merged_data['Dewormed'].map(type_mapping_Dewormed)

type_mapping_Sterilized = {1: 'Yes', 2: 'No', 3: 'Not Sure'}
merged_data['Sterilized'] = merged_data['Sterilized'].map(type_mapping_Sterilized)

#new
type_mapping_MaturitySize = {1: 'Small', 2: 'Medium', 3: 'Large', 4: 'Extra Large', 0: 'Not Specified'}
merged_data['MaturitySize'] = merged_data['MaturitySize'].map(type_mapping_MaturitySize)

type_mapping_FurLength = {1: 'Short', 2: 'Medium', 3: 'Long', 0: 'Not Specified'}
merged_data['FurLength'] = merged_data['FurLength'].map(type_mapping_FurLength)

type_mapping_Health = {1: 'Healthy', 2: 'Minor Injury', 3: 'Serious Injury', 0: 'Not Specified'}
merged_data['Health'] = merged_data['Health'].map(type_mapping_Health)


In [21]:
state_df = pd.read_csv('./state_labels.csv')
merged_data = merged_data.merge(state_df, how = 'left', left_on = 'State', 
                                right_on = 'StateID').drop(['StateID', 'State'], axis = 1)

In [22]:
merged_data.head(2)

,Type,Age,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,...,Health,Quantity,Fee,VideoAmt,Description,PhotoAmt,AdoptionSpeed,Breed1,Breed2,StateName
0,Cat,3,Male,Black,White,NA,Small,Short,No,No,...,Healthy,1,100,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,1,Tabby,NaN,Selangor
1,Cat,1,Male,Black,Brown,NA,Medium,Medium,Not Sure,Not Sure,...,Healthy,1,0,0,I just found it alone yesterday near my apartm...,2.0,1,Domestic Medium Hair,NaN,Kuala Lumpur


In [23]:
tr_data_X = merged_data[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'StateName', 'VideoAmt',
         'PhotoAmt', 'Description']]
tr_data_y = merged_data['AdoptionSpeed']

In [24]:
def col_to_str_fn(array):
    return ' | '.join([f'{col} {str(val)}' for col, val in zip(tr_data_X.columns, array)])

tr_data_X['combined'] = tr_data_X.apply(col_to_str_fn, axis = 1)

In [25]:
# tr_data_X

In [61]:
# tr_data_X.loc[464]['combined']

In [26]:
# %%capture
# pip install transformers

In [27]:
# pip show transformers

In [28]:
# import transformers
# from transformers import TFAutoModel

In [29]:
from transformers import BertTokenizer, TFAutoModel
import tensorflow as tf
import numpy as np
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model
import pandas as pd

In [ ]:
# def f1_metric(y_true, y_pred): 
#     y_pred = tf.round(y_pred) 
#     f1 = 2 * tf.reduce_sum(y_true * y_pred) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-16)
#     return f1

from tensorflow.keras.callbacks import Callback

# class MetricsLogger(Callback):
#     def __init__(self, filename):
#         super(MetricsLogger, self).__init__()
#         self.filename = filename

#     def on_epoch_end(self, epoch, logs=None):
#         with open(self.filename, 'a') as f:
#             if epoch == 0:
#                 headers = list(logs.keys())
#                 f.write(','.join(headers) + '\n')

#             metrics = [str(logs[key]) for key in logs.keys()]
#             f.write(','.join(metrics) + '\n')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
SEQ_LEN = 256 
encoded_inputs = tokenizer(tr_data_X['combined'].tolist(),
                           add_special_tokens=True,
                           padding='max_length',
                           truncation=True,
                           max_length=SEQ_LEN,
                           return_token_type_ids=False,
                           return_tensors='tf')

def map_bert(inputs, labels):
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}, labels

train_dataset = tf.data.Dataset.from_tensor_slices((encoded_inputs, tr_data_y.values))
train_dataset = train_dataset.map(map_bert)
dataset = train_dataset.shuffle(100000).batch(64)
DS_LEN = len(dataset)

SPLIT = 0.75
train_ds = dataset.take(round(DS_LEN * SPLIT))
val_ds = dataset.skip(round(DS_LEN * SPLIT))

bert = TFAutoModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), dtype=tf.int32, name='input_ids')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), dtype=tf.int32, name='attention_mask')
embeddings = bert([input_ids, mask])[0]
cls_token = embeddings[:, 0, :]
x = tf.keras.layers.BatchNormalization()(cls_token)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=75e-3)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=[
                  BinaryAccuracy(name='accuracy')
              ])

callbacks = [
    ReduceLROnPlateau(patience=3, factor=0.1, min_delta=0.001, monitor='val_loss'),
    EarlyStopping(patience=11, min_delta=0.001, monitor='val_loss'),
    ModelCheckpoint(filepath='./model/best_bert_unimodal_test.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
]

# filename = './model/metrics_log_test.csv'
# metrics_logger = MetricsLogger(filename=filename)
# callbacks.append(metrics_logger)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=callbacks
)

model.evaluate(val_ds)
model.save('./model/trained_bert_unimodal_test.h5')

172/172 [==============================] - ETA: 0s - loss: 0.8571 - accuracy: 0.4985 
Epoch 1: val_loss improved from inf to 0.69343, saving model to ./model/best_bert_unimodal_test.hdf5


/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


172/172 [==============================] - 9440s 55s/step - loss: 0.8571 - accuracy: 0.4985 - val_loss: 0.6934 - val_accuracy: 0.5017 - lr: 0.0750
Epoch 2/3
121/172 [====================>.........] - ETA: 34:18 - loss: 0.6941 - accuracy: 0.4979